In [1]:
import numpy as  np
import pandas as pd
import matplotlib.pyplot as plt
import os
import numba
import seaborn as sns
from IPython.display import display, HTML
import helperfunctions # this is my own python file
from stdev_emsd import stdev_emsd # this is my own python file

display(HTML("<style>.container { width: 90% !important; }</style>")) # Set the custom CSS to make the notebook width 90% of the window size
sns.set_context('notebook')

In [2]:
# control_tracer_nodrift.pkl files are trajectories that are already drift-subtracted
filename_list = [r"C:\Users\pguest\Desktop\pickles\Data Taken 2024-07-22, 2024-07-22_ND4_2.avicontrol_tracer_nodrift.pkl",
                          r"C:\Users\pguest\Desktop\pickles\Data Taken 2024-07-22, 2024-07-22_ND4_1.avicontrol_tracer_nodrift.pkl"]

fps_list = [1,1]
scaling_list = [1,1]

assert all(fps == fps_list[0] for fps in fps_list) # It's hard to combine emsds if these aren't all the same

In [9]:
# Create emsd dataframes with the following columns:
    # lagt = lag time, 
    # msd = ensemble mean square displacement, 
    # N = effective number of independent measurements, 
    # std_msd = standard deviation of mean square displacement
    # stderr_msd = standard error of mean square displacement
    # weight_msd = the weight to use in a weighted average of emsds

trajectories_list = [] # initialize list of dataframes
emsd_list = []
for filepath, fps, scaling in zip(filename_list, fps_list, scaling_list):
    traj = pd.read_pickle(filepath)
    trajectories_list.append(traj) # read trajectory files
    emsd_full_df = stdev_emsd(traj, mpp=scaling, fps = fps, detail = True)
    emsd_df = emsd_full_df[['lagt', 'msd', 'N', 'std_msd']]
    # Calculate standard error from standard deviation
    stderr_msd = emsd_df.std_msd / np.sqrt(emsd_df.N)
    emsd_df = emsd_df.assign(stderr_msd=stderr_msd)

    # calculate weight column for combining these files
    weight_msd = 1/(emsd_df.stderr_msd)**2
    emsd_df = emsd_df.assign(weight_msd=weight_msd)
    
    emsd_list.append(emsd_df)   

# Create big dataframe named big_df to organize all these trajectories and their emsds.
big_df = pd.DataFrame({'paths':filename_list, 
                       'files': [os.path.basename(filepath) for filepath in filename_list],   
                       'fps': fps_list, 'scaling': scaling_list, 
                       'trajectory': trajectories_list, 
                       'emsd': emsd_list})


In [10]:
big_df

,paths,files,fps,scaling,trajectory,emsd
0,C:\Users\pguest\Desktop\pickles\Data Taken 202...,"Data Taken 2024-07-22, 2024-07-22_ND4_2.avicon...",1,1,x y relative_to_fr...,lagt msd N std_...
1,C:\Users\pguest\Desktop\pickles\Data Taken 202...,"Data Taken 2024-07-22, 2024-07-22_ND4_1.avicon...",1,1,x y relative_to_fr...,lagt msd N std_...


In [11]:
big_df.emsd[0]

,lagt,msd,N,std_msd,stderr_msd,weight_msd
frame,,,,,,
1,1.0,1.052094,38338.516160,0.585594,0.002991,111799.990330
2,2.0,2.109952,25540.135799,0.614958,0.003848,67535.585878
3,3.0,3.206564,18132.829075,0.605981,0.004500,49379.520638
4,4.0,4.302206,13909.004615,0.639604,0.005423,33999.564391
5,5.0,5.418767,11240.791410,0.670774,0.006327,24982.971751
...,...,...,...,...,...,...
96,96.0,108.735754,553.277029,28.590593,1.215491,0.676856
97,97.0,109.832785,547.135690,29.580564,1.264617,0.625291
98,98.0,110.754101,541.120345,29.698713,1.276705,0.613506


In [12]:
big_df.emsd[1]

,lagt,msd,N,std_msd,stderr_msd,weight_msd
frame,,,,,,
1,1.0,1.073078,38969.071443,0.516740,0.002618,145940.240241
2,2.0,2.134723,25961.197431,0.632032,0.003923,64989.939751
3,3.0,3.228223,18432.521824,0.613926,0.004522,48904.979555
4,4.0,4.337312,14139.480837,0.701227,0.005897,28755.200450
5,5.0,5.460197,11427.541618,0.790410,0.007394,18291.461985
...,...,...,...,...,...,...
96,96.0,105.979312,565.626752,27.306254,1.148146,0.758587
97,97.0,107.255282,559.391496,27.356757,1.156663,0.747458
98,98.0,108.144764,553.283932,28.156108,1.197012,0.697915


In [ ]:
# lagt = copied
# wavmsd = weighted average
# N = added
# std_msd = weighted average
# stderr = recalculated from std and N.